In [ ]:
!pip install PyPDF2

In [ ]:
!pip install pdfplumber

In [ ]:
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import re
import pdfplumber

ModuleNotFoundError: No module named 'PyPDF2'

In [ ]:
pdf_path = "/content/Quetiapine.pdf"  # Change this to your file path

In [ ]:
with pdfplumber.open(pdf_path) as pdf:
    full_text = " ".join([page.extract_text() for page in pdf.pages if page.extract_text()])

# Normalize spaces (remove extra newlines, fix word breaks)
full_text = re.sub(r"\s+", " ", full_text)

In [ ]:
#if there is a discrepancy between the number of results in micromedex and this code, run it to find where is the problem
#full_text

In [ ]:
interaction_blocks = re.findall(r"([A-Za-z0-9\[\]\(\)\/\-\,\'\_ ]+)\s*--\s*([A-Za-z0-9\[\]\(\)\/\-\,\'\_ ]+) (Contraindicated|Major|Moderate) (Excellent|Good|Fair) (Concurrent use of [^.]*\.)", full_text)

In [ ]:
# Convert extracted interactions into a structured list
data = []
for drug1, drug2, severity, documentation, summary in interaction_blocks:
    data.append([drug1.strip(), drug2.strip(), severity.strip(), documentation.strip(), summary.strip()])

# Convert to DataFrame
df = pd.DataFrame(data, columns=["First Drug Name", "Second Drug Name", "Severity", "Documentation", "Summary"])

# Save to CSV
csv_filename = "drug_interactions_final.csv"
df.to_csv(csv_filename, index=False)

# Print final results
print(f"✅ CSV file saved: {csv_filename}")
print(f"✅ Total interactions extracted: {len(df)}")

✅ CSV file saved: drug_interactions_final.csv
✅ Total interactions extracted: 178


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pandas as pd
from collections import defaultdict

# === Load csv1 ===
csv1_path = '/content/drive/My Drive/Drug Interaction/Drugs List - IHIO/bimarestani.csv'
csv1_column = 'generic code'
csv1 = pd.read_csv(csv1_path)
values_to_search = set(csv1[csv1_column].dropna().unique())

# === Initialize containers ===
drug_to_pre_ids = defaultdict(set)
drug_to_total_cost = defaultdict(float)
drug_to_ihio_cost = defaultdict(float)
drug_occurrence_count = defaultdict(int)  # << NEW: total count of each drug

# === Process csv2 in chunks ===
csv2_path = '/content/drive/My Drive/Drug Interaction/Drugs List - IHIO/source.csv'
csv2_columns = ['pre_id', 'pres_gen_code', 'total_cost', 'IHIO_cost']
chunksize = 100_000

for chunk in pd.read_csv(csv2_path, usecols=csv2_columns, chunksize=chunksize):
    filtered = chunk[chunk['pres_gen_code'].isin(values_to_search)]

    for _, row in filtered.iterrows():
        drug = row['pres_gen_code']
        pre_id = row['pre_id']
        total_cost = row['total_cost']
        ihio_cost = row['IHIO_cost']

        drug_to_pre_ids[drug].add(pre_id)
        drug_to_total_cost[drug] += total_cost
        drug_to_ihio_cost[drug] += ihio_cost
        drug_occurrence_count[drug] += 1  # << increment for every row

# === Map results to csv1 ===
csv1['unique_pre_id_count'] = csv1[csv1_column].map(lambda x: len(drug_to_pre_ids.get(x, set())))
csv1['total_cost_sum'] = csv1[csv1_column].map(lambda x: drug_to_total_cost.get(x, 0.0))
csv1['ihio_cost_sum'] = csv1[csv1_column].map(lambda x: drug_to_ihio_cost.get(x, 0.0))
csv1['occurrence_count'] = csv1[csv1_column].map(lambda x: drug_occurrence_count.get(x, 0))

# === Show or save ===
print(csv1[['generic code', 'unique_pre_id_count', 'occurrence_count', 'total_cost_sum', 'ihio_cost_sum']])

# Optional: Save to CSV
# csv1.to_csv('/content/drive/My Drive/Drug Interaction/Drugs List - IHIO/bimarestani_with_all_stats.csv', index=False)

     generic code  unique_pre_id_count  occurrence_count  total_cost_sum  \
0              16                    0                 0               0   
1           11151                    1                 1          733400   
2              64                   34                49         2476698   
3              87                    0                 0               0   
4            1388                    0                 0               0   
..            ...                  ...               ...             ...   
266          1165                  403               440       272486000   
267           481                    0                 0               0   
268          6668                    0                 0               0   
269          2189                    0                 0               0   
270         51899                   34                37       108704036   

     ihio_cost_sum  
0                0  
1                0  
2          2351656  
3  

In [15]:
csv1.to_csv('csv1_with_counts.csv', index=False, encoding='utf-8-sig')